# Examen – Modelos de Ensamble: Árboles y Boosting

**Instrucciones**: Justifica cada respuesta de manera clara. Usa fórmulas donde aplique y sé específico en tus argumentos.

---

## Sección I: Árboles de decisión (15 puntos)

### 1. (5 pts)  
**Explica cómo se construye un árbol y qué criterio usa para decidir los splits. Explica tanto para el caso de clasificación como de regresión.**

Un modelo el cual utiliza varios niveles, donde en cada nivel existe una caractrística que se evalua a través de un umbral, va dividiendo los datos en los que son mayor o menor al umbral definido en la característica.


### Regresión

Este algoritmo usa todas las variables y los splits posibles (umbrales) y elige el umbral en cada variable que más reduzca la varianza.


$$
\text{Reducción de varianza} = \text{Varianza total} - \left( \frac{N_1}{N} \times \text{Var}(G1) + \frac{N_2}{N} \times \text{Var}(G2) \right)
$$

Varianza de cada rama ponderada por el número de gente de cada una de las ramas.

### Clasificación

De igual forma que en la regresión, el modelo usa todas las variables y prueba todos los umbrales posibles, eligiendo el que cause una mayor reducción de impureza, la cual se calcula a través del Gini o de la Entriopía.

$$
Gini = 1 - \sum p_i^2
$$

$$
Entropía = -\sum p_i \log_2 p_i
$$


$$
\text{Ganancia de Impureza} = \text{Impureza Nodo Padre} - \left( \frac{N_1}{N} \times \text{Impureza}(G1) + \frac{N_2}{N} \times \text{Impureza}(G2) \right)
$$

Impureza de cada rama ponderada por el número de personas en cada una de ellas.

### 2. (5 pts)  
**Da un ejemplo de sobreajuste en un árbol de decisión. Explica cómo se podría evitar sin necesidad de usar ensambles.**

### 3. (5 pts)  
**Si te fijas, en clase nunca hicimos escalamiento (`StandardScaler`). ¿Por qué los árboles no lo necesitan?**

El `StandardScaler` sirve para poner en una misma escala tus variables, esto es útil para modelos de KNN que usan distancias, y puede ser útil en regresiones para dar como la misma importancia a las variables, sin embargo, por la construcción de los árboles esto no es necesario ya que comparas sobre el valor real de tus datos para definir tus reglas de decisión (umbrales) que son los que más reducen la varianza. Al hacer splits sobre tus valores no es de ayuda escalarlas ya que con los valores originales se pueden establecer los splits adecuados.

---

## Sección II: Random Forest (20 puntos)

### 4. (10 pts)  
**Explica cómo funciona un Random Forest. ¿En qué se basa? ¿Por qué es una buena idea?**

### 5. (10 pts)  
**Menciona dos ventajas y dos desventajas del Random Forest comparado con un solo árbol. Sé específico, no generalices.**

---

## Sección III: Gradient Boosting (25 puntos)

### 6. (10 pts)  
**Explica, paso a paso, cómo funciona el algoritmo de **`Gradient Boosting`**. Incluye el concepto de residuales y cómo se minimiza la pérdida en cada iteración.**

### 7. (15 pts)  
**¿Cuál es la diferencia entre Gradient Boosting y Random Forest en términos de cómo combinan los árboles?** 

---


## Sección IV: XGBoost (20 puntos)

### 8. (10 pts)  
**Explica cómo XGBoost optimiza el proceso de boosting usando una expansión de Taylor de segundo orden.**

### 9. (5 pts)  
**¿Qué es el *similarity score*? ¿Qué es el *output value*? ¿De dónde salen estas fórmulas y cuál es su interpretación?**

### 10. (5 pts)  
**XGBoost y otros modelos de gradient boosting permiten evaluar la importancia de las variables con diferentes métricas: `weight` y `gain`. Explica qué representa cada una. ¿Cuál crees que es más útil para interpretar un modelo y por qué?**

---

## Sección V: LightGBM y CatBoost (15 puntos)

### 10. (5 pts)  
**Explica qué es **histogram-based splitting** y cómo lo implementa LightGBM para ganar velocidad.**

### 11. (5 pts)  
**¿Qué problema específico resuelve CatBoost respecto al manejo de variables categóricas? ¿Cómo lo hace?**

### 12. (5 pts)  
**Compara LightGBM y CatBoost: ¿cuándo usarías uno sobre el otro? Sé claro y justifica en base a tipo de datos, velocidad o precisión.**

## Sección VI: Power Analysis (5 puntos)

### 13. (5 pts)  
**¿Qué es un *power analysis* y para qué sirve? ¿En qué contexto lo hemos usado en clase y por qué es importante antes de correr un experimento?**

